In [1]:
from Crawler.Utils import get_and_unwrap
from Crawler.Data import SyllabusRegistry
from Crawler.Constants import Constants

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
html = get_and_unwrap(Constants.CMU_SYLLABUS_REGISTRY_URL.value)

In [ ]:
sr = SyllabusRegistry(html=html)

f24 = sr.semesters[0]
f24_archi = f24.departments[0]
f24_archi.get()
f24_archi_courses = f24_archi.courses

f24_archi_courses['Available Syllabi'][3].href
f24_archi_courses['Available Syllabi'][3].get()
f24_archi_courses['Available Syllabi'][3].result

In [ ]:
f24_cs = f24.departments[49]
f24_cs.get()

for course in f24_cs.courses['Available Syllabi']:
    course.get()
    analysis = course.analyze(course.result)
    if analysis == None:
        print('abnormal:', course.href)
    else:
        print(analysis) 

In [ ]:
for semester in sr.semesters:
    print(semester.name)
    for department in semester.departments:
        print(department.name)
        department.get()
        for course in department.courses['Available Syllabi']:
            course.get()
            archive = course.analyze(course.result)
            if archive == None:
                print('abnormal:', course.href)
            else:
                print(archive)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

cookie = {
    'canvas_session': 'fktJ3TEmxAjzF-f8_8y2tg+DZQDv4iGkjYEae63JMJoEmz6hO-fQItl1OsvSiGmjT1ouFM0ThLuSPigYE2J9owKz_aRECYZB8sqG-Ggt7hsncEjutwZZcPCm8AWQhIL9rs3yRWDuTkGmorQY2UKva1e9xHgjkDOESOIvdf5V57xGzFVYUrpHwa4NNy35yV3deXwiAqreQt26F1-hgvPLjdQf1K8MfqrLYkLSOizlQEwqNVYQ3_rVmwrOOU2ugnzQcmoiTNB8J-QQb0JM7_aeQ_g7st4z3eHf_bL5laKE0u56nTiNRar-CZDUHhRBLdwrRw12cdpwLuvgNyR-q4P3AApNiXquD2vEOJkzwOAZS4_PWU41cwlGfSXRMr-L1A_wp3Bm6e2P-nl2cGS1hg2QQMJIfMIAhBEd084r70eNZbFgmZts5vLOQzPEH3N28mSPu2Zfnu6brxNOYgio-DsWJvntTUIs37I_EJMV32nKmaaSSBcqHLilH_M2WsvjcPMHwCQS3SlTjniho0ml2IzMPw7omsGJz2ZDRFaik1KnzRFpwN-_kTeig1HTuSpXByD6eAFrCzcyXspHBj4KebA5XhWdiTw17tCxiB3zl1Dk1pA1QrbsgHEaELWV16xPuGcd-gjxKct_kOKZYWuQaDvbmb-JXOAbMhrZWRN1ZM22wDHfSrzU3TlPJICN0QWs3tPxE0F1hADC_cKI_u3IEVeDhDTSxRdjlj-Xj4zOyynYUPDLv2t9el8J-V0gcit1AfvYGS8ZOTCpojukeC32VvBQ1LkCP6C8p3AYkVYDZr0RJa6C669st5fcdAMhxIicDStOEOq4IxxoRKuoeHGWQjxrAkn3HonkTIwBK2y71Sv-K7VvsrXHzanGcSswpu7W5IlQgsiBt29ShCe0GsFymtQbzvgpRUG7vBGOkpvkCC0o4KByw.W6W1ANo3y0UU0YHPQaOUg-b51hM.Z2FvqQ'
}
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)
driver.get('https://httpbin.org/headers')
driver.add_cookie({
    'name': list(cookie.keys())[0], 
    'value': list(cookie.values())[0],
})
driver.get('https://canvas.cmu.edu/courses/3769/modules/items/3247653')

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
